In [2]:
pip install "numpy<2.0"


In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
pip install tqdm

In [4]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d399e2f9d7afc3a0be2c38fbd9103783fa82835a4a808d757c4a9000814796eb
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
pip install pytorch-crf


In [6]:
pip install seaborn

In [7]:
pip install nltk

In [8]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from functools import partial
import uuid
from collections import Counter
from seqeval.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
import logging
import re


In [9]:
import logging

In [ ]:
import re

In [9]:
class BiLSTMForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_labels, dropout=0.1):
        super(BiLSTMForTokenClassification, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        logits = self.fc(lstm_out)
        return logits

In [10]:
class PIIDataset(Dataset):
    def __init__(self, examples, max_length):
        self.examples = examples
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        input_ids = torch.tensor(item['input_ids'], dtype=torch.long)
        labels = torch.tensor(item['labels'], dtype=torch.long)
        attention_mask = torch.ones_like(input_ids)

        # Pad or truncate input_ids and labels to max_length
        input_ids = input_ids[:self.max_length].tolist()
        labels = labels[:self.max_length].tolist()
        attention_mask = attention_mask[:self.max_length].tolist()

        input_ids += [0] * (self.max_length - len(input_ids))
        labels += [-100] * (self.max_length - len(labels))
        attention_mask += [0] * (self.max_length - len(attention_mask))

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

In [11]:
def process_data(example, labels2int, vocab, max_length):
    tokens = example["source_text"].split()
    labels = ["O"] * len(tokens)

    for label in example["privacy_mask"]:
        start_char, end_char = label['start'], label['end']
        start_token = len(example["source_text"][:start_char].split())
        end_token = min(len(example["source_text"][:end_char].split()), len(tokens))

        label_type = label['label']
        if label_type != "CARDISSUER":  # Skip CARDISSUER labels
            if label_type.startswith("GIVENNAME"):
                label_type = "GIVENNAME"
            elif label_type.startswith("LASTNAME"):
                label_type = "LASTNAME"

            if start_token < len(labels):
                labels[start_token] = f'B-{label_type}'
                for i in range(start_token + 1, end_token):
                    if i < len(labels):
                        labels[i] = f'I-{label_type}'

    input_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens[:max_length]]
    label_ids = [labels2int[label] for label in labels[:max_length]]

    # Pad or truncate input_ids and label_ids to max_length
    input_ids = input_ids[:max_length] + [vocab['<PAD>']] * (max_length - len(input_ids))
    label_ids = label_ids[:max_length] + [labels2int['O']] * (max_length - len(label_ids))

    return {
        'input_ids': input_ids,
        'labels': label_ids
    }

In [12]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs, id2label, patience=3):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask=attention_mask)
            loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'train_loss': f"{loss.item():.4f}"})

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"\nEpoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                logits = model(input_ids, attention_mask=attention_mask)
                loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
                val_loss += loss.item()

                preds = torch.argmax(logits, dim=-1)

                for i, mask in enumerate(attention_mask):
                    pred = [id2label[p.item()] for p, m in zip(preds[i], mask) if m.item() == 1]
                    true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                    if pred and true:
                        all_preds.append(pred)
                        all_labels.append(true)

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Validation loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                model.load_state_dict(best_model)
                break

        print("Label distribution in true labels:")
        print(Counter(sum(all_labels, [])))

        print("\nLabel distribution in predictions:")
        print(Counter(sum(all_preds, [])))

        print(f"\nNumber of sequences evaluated: {len(all_labels)}")
        print(f"Number of sequences with predictions: {len(all_preds)}")
        print(f"Number of sequences with labels: {len(all_labels)}")

        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds))

    # Plot learning curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning Curve')
    plt.legend()
    plt.savefig('learning_curve.png')
    plt.close()

    return model, all_labels, all_preds

In [13]:
def plot_confusion_matrix(true_labels, pred_labels, label_list):
    # Flatten the lists of labels
    true_flat = [item for sublist in true_labels for item in sublist]
    pred_flat = [item for sublist in pred_labels for item in sublist]

    # Count occurrences of each label
    true_counts = Counter(true_flat)
    pred_counts = Counter(pred_flat)
    print("True label counts:", true_counts)
    print("Pred label counts:", pred_counts)

    # Create confusion matrix
    cm = confusion_matrix(true_flat, pred_flat, labels=label_list)

    # Plot confusion matrix
    plt.figure(figsize=(20, 16))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_list, yticklabels=label_list)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

    print(f"Confusion matrix saved as 'confusion_matrix.png'")

In [14]:
def main():
    # Define labels (excluding CARDISSUER)
    labels = [
        "BOD", "BUILDING", "CITY", "COUNTRY", "DATE", "DRIVERLICENSE",
        "EMAIL", "GEOCOORD", "GIVENNAME", "IDCARD", "IP", "LASTNAME",
        "PASS", "PASSPORT", "POSTCODE", "SECADDRESS", "SEX",
        "SOCIALNUMBER", "STATE", "STREET", "TEL", "TIME", "TITLE", "USERNAME"
    ]

    label_list = ["O"] + [f"B-{label}" for label in labels] + [f"I-{label}" for label in labels]
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {v: k for k, v in label2id.items()}

    # Load dataset
    ds = load_dataset("ai4privacy/pii-masking-300k")
    ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)

    # Remove rows with "CARDISSUER" label
    ds = ds.filter(lambda x: "CARDISSUER" not in x["mbert_bio_labels"], num_proc=4)

    # Create vocabulary
    vocab = {'<PAD>': 0, '<UNK>': 1}
    for example in ds['train']:
        for token in example['source_text'].split():
            if token not in vocab:
                vocab[token] = len(vocab)

    max_length = 512
    ds = ds.map(
        partial(process_data, labels2int=label2id, vocab=vocab, max_length=max_length),
        remove_columns=['privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'source_text'],
        num_proc=8
    )

    # Split the validation set into validation and test sets
    val_test_split = 0.3
    val_dataset = ds["validation"]
    val_dataset = val_dataset.shuffle(seed=42)
    val_size = int(len(val_dataset) * (1 - val_test_split))
    test_size = len(val_dataset) - val_size
    val_dataset, test_dataset = val_dataset.select(range(val_size)), val_dataset.select(range(val_size, len(val_dataset)))

    # Create datasets and dataloaders
    train_dataset = PIIDataset(ds["train"], max_length)
    val_dataset = PIIDataset(val_dataset, max_length)
    test_dataset = PIIDataset(test_dataset, max_length)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = BiLSTMForTokenClassification(
        vocab_size=len(vocab),
        embedding_dim=100,
        hidden_dim=128,
        num_labels=len(label_list)
    ).to(device)
    print(model.eval())

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

    # Train the model
    num_epochs = 3
    print(f"Starting training for {num_epochs} epochs...")
    model, true_labels, pred_labels = train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs, id2label)

    # Save the model
    model_save_path = f"bilstm_pii_model_{str(uuid.uuid4())}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved successfully to {model_save_path}")

    # Test the model
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(logits, dim=-1)

            for i, mask in enumerate(attention_mask):
                pred = [id2label[p.item()] for p, m in zip(preds[i], mask) if m.item() == 1]
                true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                if pred and true:
                    test_preds.append(pred)
                    test_labels.append(true)

    # Print test results
    print("\nTest Results:")
    print(classification_report(test_labels, test_preds))

    # Plot confusion matrix for test results
    plot_confusion_matrix(test_labels, test_preds, label_list)


In [15]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/177677 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/47728 [00:00<?, ? examples/s]

...


Epoch 3/3, Average training loss: 0.0266


Validation: 100%|██████████| 87/87 [02:21<00:00,  1.62s/it]


Validation loss: 0.0336
Label distribution in true labels:
Counter({'O': 2804958, 'B-TIME': 2578, 'B-LASTNAME': 2166, 'B-USERNAME': 1960, 'B-IDCARD': 1785, 'B-EMAIL': 1779, 'B-SOCIALNUMBER': 1711, 'B-DRIVERLICENSE': 1680, 'B-PASSPORT': 1673, 'B-GIVENNAME': 1672, 'B-BOD': 1644, 'B-IP': 1511, 'I-STREET': 1410, 'B-CITY': 1384, 'B-SEX': 1382, 'B-STATE': 1363, 'B-TEL': 1362, 'B-STREET': 1327, 'B-TITLE': 1321, 'B-POSTCODE': 1305, 'B-BUILDING': 1279, 'B-DATE': 1192, 'B-PASS': 1162, 'B-COUNTRY': 1105, 'I-BOD': 881, 'I-TEL': 782, 'I-TIME': 735, 'I-DATE': 684, 'B-SECADDRESS': 595, 'I-SOCIALNUMBER': 555, 'I-DRIVERLICENSE': 522, 'I-CITY': 500, 'I-SECADDRESS': 374, 'I-POSTCODE': 374, 'I-COUNTRY': 297, 'I-SEX': 191, 'B-GEOCOORD': 152, 'I-GEOCOORD': 96, 'I-PASS': 74, 'I-USERNAME': 69, 'I-LASTNAME': 60, 'I-EMAIL': 44, 'I-TITLE': 16, 'I-STATE': 15, 'I-GIVENNAME': 8, 'I-IDCARD': 6, 'I-PASSPORT': 5})

Label distribution in predictions:
Counter({'O': 2817323, 'B-IDCARD': 1799, 'B-USERNAME': 1654, 'B-TIME'

Testing: 100%|██████████| 38/38 [01:01<00:00,  1.62s/it]



Test Results:
               precision    recall  f1-score   support

          BOD       0.77      0.58      0.66       657
     BUILDING       0.83      0.70      0.76       547
         CITY       0.68      0.70      0.69       598
      COUNTRY       0.92      0.83      0.87       436
         DATE       0.68      0.31      0.43       480
DRIVERLICENSE       0.65      0.39      0.49       692
        EMAIL       0.75      0.54      0.63       739
     GEOCOORD       0.20      0.03      0.05        64
    GIVENNAME       0.58      0.17      0.27       651
       IDCARD       0.51      0.47      0.49       812
           IP       0.84      0.36      0.51       624
     LASTNAME       0.62      0.25      0.35       974
         PASS       0.87      0.46      0.60       435
     PASSPORT       0.76      0.46      0.58       718
     POSTCODE       0.77      0.69      0.73       570
   SECADDRESS       0.87      0.72      0.79       264
          SEX       0.89      0.74      0.81     